In [1]:
using LinearAlgebra
using TensorOperations
using ITensors, ITensorMPS
using Plots

In [10]:
N = 10
s = siteinds(3,N)
chi = 20
mps = random_mps(s;linkdims=chi)
println("Right canonical MPS bond structure $mps")

Right canonical MPS bond structure MPS
[1] ((dim=3|id=320|"Site,n=1"), (dim=20|id=714|"Link,l=1"))
[2] ((dim=20|id=714|"Link,l=1"), (dim=3|id=431|"Site,n=2"), (dim=20|id=449|"Link,l=2"))
[3] ((dim=20|id=449|"Link,l=2"), (dim=3|id=201|"Site,n=3"), (dim=20|id=523|"Link,l=3"))
[4] ((dim=20|id=523|"Link,l=3"), (dim=3|id=513|"Site,n=4"), (dim=20|id=115|"Link,l=4"))
[5] ((dim=20|id=115|"Link,l=4"), (dim=3|id=111|"Site,n=5"), (dim=20|id=421|"Link,l=5"))
[6] ((dim=20|id=421|"Link,l=5"), (dim=3|id=179|"Site,n=6"), (dim=20|id=862|"Link,l=6"))
[7] ((dim=20|id=862|"Link,l=6"), (dim=3|id=721|"Site,n=7"), (dim=20|id=741|"Link,l=7"))
[8] ((dim=20|id=741|"Link,l=7"), (dim=3|id=702|"Site,n=8"), (dim=9|id=694|"Link,l=8"))
[9] ((dim=9|id=694|"Link,l=8"), (dim=3|id=295|"Site,n=9"), (dim=3|id=733|"Link,l=9"))
[10] ((dim=3|id=733|"Link,l=9"), (dim=3|id=108|"Site,n=10"))



# MPOs of Hamiltonians

$\quad$ In this section, we show how to build the MPO of a model Hamiltonian, following the strategy explained in the manuscript. For concreteness, we focus on Hamiltonians of 1D quantum spin chains with open boundary conditions. A general representation of such an Hamiltonian MPO is depicted below, where $N$ is the number of sites in the chain and $d$ is the corresponding local dimension ($d=2s+1$ for a spin-$s$ system).

<img src="./img/MPO_Hamiltonian.png" width="50%" style="display: block; margin-left: auto; margin-right: auto;"/>
<br>

For clarity purposes, we consider four different examples, which should illustrate how to do it for a general model.

## i) XY model

$\quad$ The Hamiltonian of an XY open-ended chain is given by
$$
\hat{\mathcal{H}}_\text{XY} = -\sum_{l=0}^{N-2} \left( \hat{\sigma}^x_l \hat{\sigma}^x_{l+1} + \hat{\sigma}^y_l \hat{\sigma}^y_{l+1} \right),
$$
where $N$ is the number of $s=1/2$ spins and $\hat{\sigma}^{x/y}_l$ are the $x$/$y$ spin-1/2 operators at site $l$.

$\quad$ Straightforward manipulation gives
$$
\hat{\mathcal{H}}_\text{XY} = -\frac{1}{2} \sum_{l=0}^{N-2} \left( \hat{\sigma}^+_l \hat{\sigma}^-_{l+1} + \hat{\sigma}^-_l \hat{\sigma}^+_{l+1} \right),
$$
where $\hat{\sigma}^\pm_l = \hat{\sigma}^x_l \pm \mathrm{i} \hat{\sigma}^y_l$ are the spin ladder operators.

In [17]:
N = 5
s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N-1)
    os += -0.5, "S+", j, "S-", j+1
    os += -0.5, "S-", j, "S+", j+1
end

In [18]:
H_mpo = MPO(os, s)

# Setup DMRG sweeps
sweeps = Sweeps(5)
setmaxdim!(sweeps, 10, 20, 100, 100, 200)
setcutoff!(sweeps, 1E-10)

# Use the same site indices for the random MPS
psi0 = randomMPS(s, 20)

# Run DMRG to find ground state
energy, psi = dmrg(H_mpo, psi0, sweeps)
println("Ground state energy from DMRG = $energy")

After sweep 1 energy=-1.3660143088205567  maxlinkdim=4 maxerr=6.37E-17 time=8.531
After sweep 2 energy=-1.3660254037076882  maxlinkdim=4 maxerr=1.42E-15 time=0.011
After sweep 3 energy=-1.3660254037844384  maxlinkdim=4 maxerr=1.17E-15 time=0.002
After sweep 4 energy=-1.3660254037844397  maxlinkdim=4 maxerr=1.36E-15 time=0.002
After sweep 5 energy=-1.3660254037844388  maxlinkdim=4 maxerr=1.31E-15 time=0.004
Ground state energy from DMRG = -1.3660254037844388
